<a href="https://colab.research.google.com/github/lukekolbe/AL-in-CreditScoring/blob/main/result_processor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prep Runtime

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [3]:
#!pip freeze

In [4]:
#!pip uninstall matplotlib -y

In [5]:
!pip install pingouin
!pip install scikit_posthocs
#!pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 183 kB 7.4 MB/s 
     |████████████████████████████████| 38.1 MB 1.1 MB/s 
     |████████████████████████████████| 9.8 MB 36.9 MB/s 
  Created wheel for pingouin: filename=pingouin-0.5.1-py3-none-any.whl size=194002 sha256=525e826858df3030e79eb57515cfa8168d345801bf94afdc64e2419ec1229a4f
  Stored in directory: /root/.cache/pip/wheels/74/a7/b6/5b5f03b0395fdada1f45d3aed76e7da48873a02805545ef652
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7048 sha256=fafb69b87d35f5075c597331df8fa9bc43705ed7e235f4c407732d3ee6638a6c
  Stored in directory: /root/.cache/pip/wheels/d6/64/cd/32819b511a488e4993f2fab909a95330289c3f4e0f6ef4676d
Successfully built pingouin littleutils
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
  Attempting

In [6]:
############ LIBRARIES

import os
import time
import datetime
import random
import pickle
import re
import copy
import gc
import sys
import json

gc.enable()

import warnings
warnings.filterwarnings('ignore')

import pingouin as pg
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
plt.style.use('default')
%matplotlib inline
import seaborn as sns

import scipy.stats
from collections import namedtuple
import scikit_posthocs as sp

In [7]:
############ RANDOMNESS
# seed function
def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    
# set seed
seed = 30
seed_everything(seed)

In [8]:
os.chdir('/gdrive/My Drive/ACTIVE LEARNING THESIS/')

# Helper Functions

In [9]:

'''
  This function is an enhanced version of the Friedman Test found in the popular scipy package.
  The added functionality computes z-values (pivotal quantities) that are needed to run the Holm-Test, i.e.
  pairwise comparisons of each model to the control model, with p-value adjustment.
  The formula for the z-statistic can be found in DEMSAR (2006).
'''

#FriedmanchisquareResult = namedtuple('FriedmanchisquareResult',
#                                     ('statistic', 'pvalue'))

def friedmanchisquare2(*args):
    """Compute the Friedman test for repeated measurements.
    The Friedman test tests the null hypothesis that repeated measurements of
    the same individuals have the same distribution.  It is often used
    to test for consistency among measurements obtained in different ways.
    For example, if two measurement techniques are used on the same set of
    individuals, the Friedman test can be used to determine if the two
    measurement techniques are consistent.
    Parameters
    ----------
    measurements1, measurements2, measurements3... : array_like
        Arrays of measurements.  All of the arrays must have the same number
        of elements.  At least 3 sets of measurements must be given.
    Returns
    -------
    statistic : float
        The test statistic, correcting for ties.
    pvalue : float
        The associated p-value assuming that the test statistic has a chi
        squared distribution.
    
    Notes
    -----
    Due to the assumption that the test statistic has a chi squared
    distribution, the p-value is only reliable for n > 10 and more than
    6 repeated measurements.
    References
    ----------
    .. [1] https://en.wikipedia.org/wiki/Friedman_test
    """

    k = len(args)
    if k < 3:
        raise ValueError('At least 3 sets of measurements must be given '
                         'for Friedman test, got {}.'.format(k))

    n = len(args[0])
    for i in range(1, k):
        if len(args[i]) != n:
            raise ValueError('Unequal N in friedmanchisquare.  Aborting.')

    # Rank data
    data = np.vstack(args).T
    data = data.astype(float)
    for i in range(len(data)):
        data[i] = scipy.stats.rankdata(data[i])

    # Handle ties
    ties = 0
    for d in data:
        replist, repnum = scipy.stats.find_repeats(np.array(d))
        for t in repnum:
            ties += t * (t*t - 1)
    c = 1 - ties / (k*(k*k - 1)*n)
    

    ssbn = np.sum(data.sum(axis=0)**2)
    chisq = (12.0 / (k*n*(k+1)) * ssbn - 3*n*(k+1)) / c #test statistic
    pvalue = scipy.stats.distributions.chi2.sf(chisq, k - 1) #p-value

    ########## ADDED CUSTOM FUNCTIONALITY TO RUN HOLM TEST ##########
    #print('data from function', data)
    rankings_avg = [np.mean([case[j] for case in data]) for j in range(k)]
    rankings_cmp = [r/np.sqrt(k*(k+1)/(6.*n)) for r in rankings_avg] #test statistics (z-value) for each method (DEMSAR 2006)
    ##########

    return chisq, pvalue, data, rankings_avg, rankings_cmp

In [10]:
# ALTERNATIVE VERSION OF FRIEDMAN TEST
# APPLIES IMAN-DAVENPORT CORRECTION TO P-VALUES

def friedman_test(*args):

    """
        Performs a Friedman ranking test.
        Tests the hypothesis that in a set of k dependent samples groups (where k >= 2) at least two of the groups represent populations with different median values.
        
        Parameters
        ----------
        sample1, sample2, ... : array_like
            The sample measurements for each group.
            
        Returns
        -------
        F-value : float
            The computed F-value of the test.
        p-value : float
            The associated p-value from the F-distribution.
        rankings : array_like
            The ranking for each group.
        pivots : array_like
            The pivotal quantities for each group.
            
        References
        ----------
        M. Friedman, The use of ranks to avoid the assumption of normality implicit in the analysis of variance, Journal of the American Statistical Association 32 (1937) 674–701.
        D.J. Sheskin, Handbook of parametric and nonparametric statistical procedures. crc Press, 2003, Test 25: The Friedman Two-Way Analysis of Variance by Ranks
    """

    k = len(args)
    if k < 2: raise ValueError('Less than 2 levels')
    n = len(args[0])
    if len(set([len(v) for v in args])) != 1: raise ValueError('Unequal number of samples')

    rankings = []
    for i in range(n):
        row = [col[i] for col in args]
        row_sort = sorted(row)
        rankings.append([row_sort.index(v) + 1 + (row_sort.count(v)-1)/2. for v in row])

    rankings_avg = [np.mean([case[j] for case in rankings]) for j in range(k)]
    rankings_cmp = [r/np.sqrt(k*(k+1)/(6.*n)) for r in rankings_avg] #see demsar (2006)

    chi2 = ((12*n)/float((k*(k+1))))*((sum(r**2 for r in rankings_avg))-((k*(k+1)**2)/float(4))) #(uncorrected) chi-square test statistic
    iman_davenport = ((n-1)*chi2)/float((n*(k-1)-chi2)) #corrected test-statistic

    p_value = 1 - scipy.stats.f.cdf(iman_davenport, k-1, (k-1)*(n-1)) #iman-davenport corrected p-value
    
    ###########
    p_value_uncorrected = scipy.stats.distributions.chi2.sf(chi2, k - 1)
    ###########

    return iman_davenport,p_value, chi2, p_value_uncorrected, rankings, rankings_avg, rankings_cmp

In [11]:
def finner_test(pivots, control=None):
    """
        Performs a Finner post-hoc test using the pivot quantities obtained by a ranking test.
        Tests the hypothesis that the ranking of the control method is different to each of the other methods.
        
        Parameters
        ----------
        pivots : dictionary_like
            A dictionary with format 'groupname':'pivotal quantity' 
        control : string optional
            The name of the control method,  default the group with minimum ranking
            
        Returns
        ----------
        comparions : array-like
            Strings identifier of each comparison with format 'group_i vs group_j'
        Z-values : array-like
            The computed Z-value statistic for each comparison.
        p-values : array-like
            The associated p-value from the Z-distribution wich depends on the index of the comparison
        adj_p-values : array-like
            The associated adjusted p-values wich can be compared with a significance level
            
        References
        ----------
        H. Finner, On a monotonicity problem in step-down multiple test procedures, Journal of the American Statistical Association 88 (1993) 920–923.
    """
    
    k = len(pivots)
    values = list(pivots.values())
    keys = list(pivots.keys())
    if not control :
        control_i = values.index(min(values))
    else:
        control_i = keys.index(control)

    comparisons = [keys[control_i] + " vs " + keys[i] for i in range(k) if i != control_i]
    z_values = [abs(values[control_i] - values[i]) for i in range(k) if i != control_i]
    p_values = [2*(1-scipy.stats.norm.cdf(abs(z))) for z in z_values]
    # Sort values by p_value so that p_0 < p_1
    p_values, z_values, comparisons= map(list, zip(*sorted(zip(p_values, z_values, comparisons), key=lambda t: t[0])))
    adj_p_values = [min(max(1-(1-p_values[j])**((k-1)/float(j+1)) for j in range(i+1)), 1) for i in range(k-1)]
    
    return comparisons, z_values, p_values, adj_p_values

In [12]:
import itertools as it

def holm_test(pivots, control=None):
    """
        Performs a Holm post-hoc test using the pivot quantities obtained by a ranking test.
        Tests the hypothesis that the ranking of the control method is different to each of the other methods.
        
        Parameters
        ----------
        pivots : dictionary_like
            A dictionary with format 'groupname':'pivotal quantity' 
        control : string optional
            The name of the control method (one vs all), default None (all vs all) 
            
        Returns
        ----------
        comparions : array-like
            Strings identifier of each comparison with format 'group_i vs group_j'
        Z-values : array-like
            The computed Z-value statistic for each comparison.
        p-values : array-like
            The associated p-value from the Z-distribution wich depends on the index of the comparison
        adj_p-values : array-like
            The associated adjusted p-values wich can be compared with a significance level
            
        References
        ----------
        O.J. S. Holm, A simple sequentially rejective multiple test procedure, Scandinavian Journal of Statistics 6 (1979) 65–70.
    """
    k = len(pivots.keys())
    values = list(pivots.values())
    keys = list(pivots.keys())
    if not control :
        control_i = values.index(min(values))
    else:
        control_i = keys.index(control)

    comparisons = [keys[control_i] + " vs " + keys[i] for i in range(k) if i != control_i]
    z_values = [abs(values[control_i] - values[i]) for i in range(k) if i != control_i]
    p_values = [2*(1-scipy.stats.norm.cdf(abs(z))) for z in z_values]
    
    # Sort values by p_value so that p_0 < p_1
    p_values, z_values, comparisons = map(list, zip(*sorted(zip(p_values, z_values, comparisons), key=lambda t: t[0])))
    adj_p_values = [min(max((k-(j+1))*p_values[j] for j in range(i+1)), 1) for i in range(k-1)]
    
    return comparisons, z_values, p_values, adj_p_values

In [13]:
def bonferroni_dunn_test(pivots, control=None):
    """
        Performs a Bonferroni-Dunn post-hoc test using the pivot quantities obtained by a ranking test.
        Tests the hypothesis that the ranking of the control method is different to each of the other methods.
        
        Parameters
        ----------
        pivots : dictionary_like
            A dictionary with format 'groupname':'pivotal quantity' 
        control : string optional
            The name of the control method (one vs all), default None (all vs all) 
            
        Returns
        ----------
        comparions : array-like
            Strings identifier of each comparison with format 'group_i vs group_j'
        Z-values : array-like
            The computed Z-value statistic for each comparison.
        p-values : array-like
            The associated p-value from the Z-distribution wich depends on the index of the comparison
        adj_p-values : array-like
            The associated adjusted p-values wich can be compared with a significance level
            
        References
        ----------
        O.J. Dunn, Multiple comparisons among means, Journal of the American Statistical Association 56 (1961) 52–64.
    """
    k = len(pivots)
    values = list(pivots.values())
    keys = list(pivots.keys())
    if not control :
        control_i = values.index(min(values))
    else:
        control_i = keys.index(control)

    comparisons = [keys[control_i] + " vs " + keys[i] for i in range(k) if i != control_i]
    z_values = [abs(values[control_i] - values[i]) for i in range(k) if i != control_i]
    p_values = [2*(1-scipy.stats.norm.cdf(abs(z))) for z in z_values]
    # Sort values by p_value so that p_0 < p_1
    p_values, z_values, comparisons = map(list, zip(*sorted(zip(p_values, z_values, comparisons), key=lambda t: t[0])))
    adj_p_values = [min((k-1)*p_value,1) for p_value in p_values]
    
    return comparisons, z_values, p_values, adj_p_values

In [14]:
# Helper functions for performing the statistical tests
def generate_scores(method, method_args, data, labels):
    pairwise_scores = method(data, **method_args) # Matrix for all pairwise comaprisons
    pairwise_scores.set_axis(labels, axis='columns', inplace=True) # Label the cols
    pairwise_scores.set_axis(labels, axis='rows', inplace=True) # Label the rows, note: same label as pairwise combinations
    return pairwise_scores

def pairwise_plotter(scores):
    # Pretty plot of significance
    heatmap_args = {'linewidths': 0.25, 'linecolor': '0.5', 'square': True,
                    'cbar_ax_bbox': [0.80, 0.35, 0.04, 0.3]}

    sp.sign_plot(scores, **heatmap_args)

# Load results as saved during computation

## SCORE METRICS

In [15]:
#load result files as specified, store in one large dictionary
dataset_list = ["bene2", "bene1_nobins","gmsc", "uk", "lendingclub", "hmeq", "australian", "german", "thomas", "pakdd"]
ratios_list = [0.1,0.2,0.3,0.4,0.5] #,0.2,0.3,0.4,0.5
cost_mat_list = [None] #only one cost setup was used, others are discarded due to unreliable results and computation issues
weights_list = ['wF', 'wT']
results_dict = {'wT': {}, 'wF': {}} #load results for parameter weights True/False separately

for w in weights_list:
  for d in dataset_list:
    results_dict[w][d] = {}
    for r in ratios_list:
      for c in cost_mat_list:
        if w == 'wT':
          filename = f"{d}__rounds-5_max_gens-10_AL_acc_rate-{r}_weights-True_cost_matrix-{c}_do_thres-tuned"
        else:
          filename = f"{d}__rounds-5_max_gens-10_AL_acc_rate-{r}_weights-False_cost_matrix-{c}_do_thres-tuned"
          
        filename = filename.replace('.', '-')
        filename = filename.replace('[', '')
        filename = filename.replace(']', '')
        filename = filename.replace(',', '-')
        filename = filename.replace(' ', '')

        infile = open(f'model_results/{d}/{filename}','rb')

        rname = f'r{r}'.replace('.', '')

        results_dict[w][d][rname] = pickle.load(infile)

        infile.close()
        #if fname not in configs_list:
        #  configs_list.append(fname)

#print(configs_list)

In [30]:
results_dict['wT']['uk']['r04']['score']['average']

,generation,n_accepts,n_rejects,bad_ratio_accepts,bad_ratio_rejects,n_score_accepts,n_AL_selects,bad_ratio_score_accepts,bad_ratio_AL_selects,roc_auc,brier,h-measure,average_precision,balanced_accuracy,pcc,precision,recall,f1,fpr,fnr,tn,fp,fn,tp
0,0.0,NaN,NaN,0.04120,0.03976,NaN,NaN,NaN,NaN,0.665934,0.038477,0.098402,0.080238,0.616615,0.489425,0.065288,0.590000,0.117221,0.356771,0.410000,3705.0,2055.0,98.4,141.6
1,1.0,827.0,173.0,0.03516,0.06850,196.4,130.6,0.01576,0.04026,0.688893,0.038319,0.118638,0.088162,0.634705,0.456539,0.066319,0.658333,0.120363,0.388924,0.341667,3519.8,2240.2,82.0,158.0
2,2.0,1127.4,372.6,0.03182,0.06744,376.8,250.6,0.01516,0.03750,0.693977,0.038338,0.124531,0.089314,0.645104,0.453389,0.068060,0.683333,0.123653,0.393125,0.316667,3495.6,2264.4,76.0,164.0
3,3.0,1427.0,573.0,0.03014,0.06610,556.6,370.4,0.01412,0.03874,0.704343,0.038293,0.134472,0.093751,0.648160,0.447494,0.068082,0.695000,0.123939,0.398681,0.305000,3463.6,2296.4,73.2,166.8
4,4.0,1722.4,777.6,0.02750,0.06902,734.0,488.4,0.01284,0.03506,0.707413,0.038233,0.139494,0.095411,0.652760,0.460118,0.069799,0.691667,0.126740,0.386146,0.308333,3535.8,2224.2,74.0,166.0
5,5.0,2021.0,979.0,0.02592,0.07030,913.2,607.8,0.01182,0.03420,0.714296,0.038214,0.145548,0.097769,0.656736,0.463674,0.070781,0.696667,0.128417,0.383194,0.303333,3552.8,2207.2,72.8,167.2
6,6.0,2314.2,1185.8,0.02594,0.06846,1089.2,725.0,0.01194,0.03618,0.715932,0.038256,0.149107,0.098536,0.661076,0.455899,0.070896,0.713333,0.128900,0.391181,0.286667,3506.8,2253.2,68.8,171.2
7,7.0,2607.2,1392.8,0.02498,0.06922,1265.2,842.0,0.01120,0.03582,0.719342,0.038188,0.151049,0.099975,0.664740,0.457553,0.071841,0.720000,0.130504,0.390521,0.280000,3510.6,2249.4,67.2,172.8
8,8.0,2900.6,1599.4,0.02442,0.06916,1441.4,959.2,0.01092,0.03574,0.720463,0.038262,0.152377,0.099732,0.662170,0.453565,0.071047,0.718333,0.129201,0.393993,0.281667,3490.6,2269.4,67.6,172.4
9,9.0,3194.0,1806.0,0.02398,0.06918,1617.6,1076.4,0.01084,0.03554,0.724194,0.038173,0.156084,0.100627,0.666962,0.456082,0.072116,0.725833,0.131079,0.391910,0.274167,3502.6,2257.4,65.8,174.2


## COST METRICS

In [16]:
#load result files as specified, store in one large dictionary
dataset_list = ["bene2", "bene1_nobins","gmsc", "uk", "lendingclub", "hmeq", "australian", "german", "thomas", "pakdd"]
cost_dict = {'wT': {}, 'wF': {}} #load results for parameter weights True/False separately
#configs_list = []

for w in weights_list:
  for d in dataset_list:
    cost_dict[w][d] = {}
    for r in ratios_list:
      for c in cost_mat_list:
        if w == 'wT':
          filename = f"{d}__rounds-5_max_gens-10_AL_acc_rate-{r}_weights-True_cost_matrix-{c}_do_thres-tuned"
        else:
          filename = f"{d}__rounds-5_max_gens-10_AL_acc_rate-{r}_weights-False_cost_matrix-{c}_do_thres-tuned"
          
        filename = filename.replace('.', '-')
        filename = filename.replace('[', '')
        filename = filename.replace(']', '')
        filename = filename.replace(',', '-')
        filename = filename.replace(' ', '')

        infile = open(f'model_cost/{d}/{filename}','rb')

        rname = f'r{r}'.replace('.', '')

        cost_dict[w][d][rname] = pickle.load(infile)

        infile.close()
        #if fname not in configs_list:
        #  configs_list.append(fname)

#print(configs_list)

In [ ]:
cost_dict['wT']['bene2']['r05']['bmdr']['average']

,generation,gen_internal_cost,gen_internal_cpl,total_internal_cost,total_internal_cpl,model_internal_cost,model_internal_cpl,external_cost,external_cpl
0,0.0,340.200000,0.680400,340.200000,0.680400,0.000000,NaN,635.200000,0.441725
1,1.0,285.533333,0.571067,625.733333,0.625733,285.533333,0.571067,611.133333,0.424988
2,2.0,271.466667,0.542933,897.200000,0.598133,557.000000,0.557000,599.400000,0.416829
3,3.0,285.333333,0.570667,1182.533333,0.591267,842.333333,0.561556,587.266667,0.408391
4,4.0,276.200000,0.552400,1458.733333,0.583493,1118.533333,0.559267,587.533333,0.408577
5,5.0,275.000000,0.550000,1733.733333,0.577911,1393.533333,0.557413,578.133333,0.402040
6,6.0,285.266667,0.570533,2019.000000,0.576857,1678.800000,0.559600,582.800000,0.405285
7,7.0,282.333333,0.564667,2301.333333,0.575333,1961.133333,0.560324,578.600000,0.402364
8,8.0,298.333333,0.596667,2599.666667,0.577704,2259.466667,0.564867,582.600000,0.405146
9,9.0,292.866667,0.585733,2892.533333,0.578507,2552.333333,0.567185,582.333333,0.404961


## Build table template to fill with aggregated results

In [17]:
# ALT: EVERY PARAMETER COMBINATION AS OWN CLASSIFER
#model_list = ['oracle','score', 'random', 'unc', 'qbc', 'dw', 'cors', 'density', 'eer', 'lal', 'quire', 'bmdr', 'spal']
model_list = ['score', 'random', 'unc', 'qbc', 'dw', 'cors', 'density', 'eer', 'lal', 'quire', 'bmdr', 'spal']
performance_metrics = ['roc_auc', 'brier', 'h-measure', 'pcc', 'balanced_accuracy', 'f1','average_precision', 'precision', 'recall', 
                       'fpr', 'fnr', 'fn', 'fp', 'tp','tn', 'bad_ratio_accepts', 'bad_ratio_rejects', 'bad_ratio_AL_selects', 'bad_ratio_score_accepts'
]
cost_metrics = ['total_internal_cost','total_internal_cpl','external_cost','external_cpl']

# built list of column names
c_names = model_list

#column_names = ['score']
column_names = []

index_names = []

#################################
for m in model_list:
  for r in ["r01","r02","r03","r04","r05"]:
    for w in weights_list:
      column_names.append(f'{m}_{w}_{r}')
  #if m != 'score':
    #column_names.append(f'{m}_r05')
#################################

for d in dataset_list:
  for s in performance_metrics:
    for g in range(10):
      index_names.append(f'{d}_{s}_g{g+1}')
  for c in cost_metrics:
    for g in range(10):
      index_names.append(f'{d}_{c}_g{g+1}')

results_template = pd.DataFrame(columns=column_names, index=index_names)
#results_template

## Fill in results

In [18]:
# fill in values from results data
performance = copy.deepcopy(results_template)

for w in ['wT', 'wF']:
  for r in ["r01","r02","r03","r04","r05"]:
    for s in performance_metrics:
      for d in dataset_list:
        for g in range(10):
          for m in model_list:
            #print('RESULTS',w,r,s,d,g,m)
            performance.at[f'{d}_{s}_g{g+1}', f'{m}_{w}_{r}'] = results_dict[w][d][r][m]['average'][s][g+1]

    for c in cost_metrics:
      for d in dataset_list:
        for g in range(10):
          for m in model_list:
            #print('COST',w,r,c,d,g,m)

            performance.at[f'{d}_{c}_g{g+1}', f'{m}_{w}_{r}'] = cost_dict[w][d][r][m]['average'][c][g+1]


model_list = ['score', 'random', 'unc', 'qbc', 'dw', 'cors', 'density', 'eer', 'lal', 'quire', 'bmdr', 'spal']

namelist=[]
for d in dataset_list:
  for g in range(10):
    namelist.append(f'{d}_total_cost_g{g+1}')
  
data = performance.loc[performance.index.str.contains('total_internal_cost')].values + performance.loc[performance.index.str.contains('external_cost')].values
total_cost = pd.DataFrame(data,index=namelist, columns = performance.columns)

performance = pd.concat((performance, total_cost))



##################################

#for m in model_list:
  #if m == 'score':
    #performance.loc[:,'score'] = performance.filter(regex=f'{m}_').mean(axis=1)
  #else:
    #performance.loc[:,f'{m}_r05'] = performance.filter(regex=f'{m}_.*_r05').mean(axis=1) 
    #no need to differentiate between wT and wF when r = 0.5, since the dynamic (inversely proportional) sample weights will all be 1
    #(no difference between score and AL-selections in terms of sample weights)

'''#remove redundant columns (score model and any wT_r05 or wF_r05) before ranking models in order to not dilute ranks
for m in model_list:
  for w in ['wT', 'wF']:
    if m == 'score':
      for r in ["r01","r02","r03","r04","r05"]:
        performance = performance.drop(f'{m}_{w}_{r}', axis=1)
    else:    
      performance = performance.drop(f'{m}_{w}_r05', axis=1)

#build ranks
performance_ranks = performance.rank(ascending=False, axis="columns")
##invert ranks for metrics where less is better
performance_ranks.loc[performance_ranks.index.str.contains('brier|cost|cpl')] = performance.loc[performance.index.str.contains('brier|cost|cpl')].rank(ascending=True, axis="columns")

# make sure each score column has same values (re-adds columns, all with the same values and ranks)
for m in model_list:
  for w in ['wT', 'wF']:
    if m == 'score':
      for r in ["r01","r02","r03","r04","r05"]:
        performance[f'{m}_{w}_{r}'] = performance.loc[:,f'{m}']
        performance_ranks[f'{m}_{w}_{r}'] = performance_ranks.loc[:,f'{m}']
    else:
      performance[f'{m}_{w}_r05'] = performance.loc[:,f'{m}_r05']
      performance_ranks[f'{m}_{w}_r05'] = performance_ranks.loc[:,f'{m}_r05']'''


#performance = performance.reindex(sorted(performance.columns), axis=1)
#performance_ranks = performance_ranks.reindex(sorted(performance_ranks.columns), axis=1)

#performance_ranks.loc['mean_ranks']=(performance_ranks.loc[~performance_ranks.index.str.contains('internal')].mean())
#performance_ranks.loc['cost_ranks']=(performance_ranks.loc[performance_ranks.index.str.contains('external_cpl')].mean())

'#remove redundant columns (score model and any wT_r05 or wF_r05) before ranking models in order to not dilute ranks\nfor m in model_list:\n  for w in [\'wT\', \'wF\']:\n    if m == \'score\':\n      for r in ["r01","r02","r03","r04","r05"]:\n        performance = performance.drop(f\'{m}_{w}_{r}\', axis=1)\n    else:    \n      performance = performance.drop(f\'{m}_{w}_r05\', axis=1)\n\n#build ranks\nperformance_ranks = performance.rank(ascending=False, axis="columns")\n##invert ranks for metrics where less is better\nperformance_ranks.loc[performance_ranks.index.str.contains(\'brier|cost|cpl\')] = performance.loc[performance.index.str.contains(\'brier|cost|cpl\')].rank(ascending=True, axis="columns")\n\n# make sure each score column has same values (re-adds columns, all with the same values and ranks)\nfor m in model_list:\n  for w in [\'wT\', \'wF\']:\n    if m == \'score\':\n      for r in ["r01","r02","r03","r04","r05"]:\n        performance[f\'{m}_{w}_{r}\'] = performance.loc[:,f\

In [31]:
performance

,score_wF_r01,score_wT_r01,score_wF_r02,score_wT_r02,score_wF_r03,score_wT_r03,score_wF_r04,score_wT_r04,score_wF_r05,score_wT_r05,random_wF_r01,random_wT_r01,random_wF_r02,random_wT_r02,random_wF_r03,random_wT_r03,random_wF_r04,random_wT_r04,random_wF_r05,random_wT_r05,unc_wF_r01,unc_wT_r01,unc_wF_r02,unc_wT_r02,unc_wF_r03,unc_wT_r03,unc_wF_r04,unc_wT_r04,unc_wF_r05,unc_wT_r05,qbc_wF_r01,qbc_wT_r01,qbc_wF_r02,qbc_wT_r02,qbc_wF_r03,qbc_wT_r03,qbc_wF_r04,qbc_wT_r04,qbc_wF_r05,qbc_wT_r05,dw_wF_r01,dw_wT_r01,dw_wF_r02,dw_wT_r02,dw_wF_r03,dw_wT_r03,dw_wF_r04,dw_wT_r04,dw_wF_r05,dw_wT_r05,cors_wF_r01,cors_wT_r01,cors_wF_r02,cors_wT_r02,cors_wF_r03,cors_wT_r03,cors_wF_r04,cors_wT_r04,cors_wF_r05,cors_wT_r05,density_wF_r01,density_wT_r01,density_wF_r02,density_wT_r02,density_wF_r03,density_wT_r03,density_wF_r04,density_wT_r04,density_wF_r05,density_wT_r05,eer_wF_r01,eer_wT_r01,eer_wF_r02,eer_wT_r02,eer_wF_r03,eer_wT_r03,eer_wF_r04,eer_wT_r04,eer_wF_r05,eer_wT_r05,lal_wF_r01,lal_wT_r01,lal_wF_r02,lal_wT_r02,lal_wF_r03,lal_wT_r03,lal_wF_r04,lal_wT_r04,lal_wF_r05,lal_wT_r05,quire_wF_r01,quire_wT_r01,quire_wF_r02,quire_wT_r02,quire_wF_r03,quire_wT_r03,quire_wF_r04,quire_wT_r04,quire_wF_r05,quire_wT_r05,bmdr_wF_r01,bmdr_wT_r01,bmdr_wF_r02,bmdr_wT_r02,bmdr_wF_r03,bmdr_wT_r03,bmdr_wF_r04,bmdr_wT_r04,bmdr_wF_r05,bmdr_wT_r05,spal_wF_r01,spal_wT_r01,spal_wF_r02,spal_wT_r02,spal_wF_r03,spal_wT_r03,spal_wF_r04,spal_wT_r04,spal_wF_r05,spal_wT_r05
bene2_roc_auc_g1,0.770134,0.770134,0.770134,0.770134,0.770134,0.770134,0.770134,0.770134,0.770082,0.770082,0.767747,0.75766,0.765351,0.762809,0.765912,0.765291,0.768434,0.770053,0.766735,0.766735,0.767798,0.750652,0.766122,0.758728,0.76472,0.761167,0.7629,0.761408,0.763335,0.763335,0.767436,0.747718,0.768637,0.766165,0.766063,0.765882,0.766648,0.767638,0.76609,0.76609,0.768474,0.755805,0.767421,0.761677,0.768591,0.767251,0.765573,0.765123,0.765673,0.765673,0.768265,0.762062,0.76813,0.768763,0.767404,0.769437,0.766213,0.767252,0.767408,0.767408,0.769048,0.759986,0.768379,0.76644,0.766077,0.766159,0.764996,0.765659,0.766311,0.766311,0.767729,0.759596,0.7666,0.75995,0.765622,0.763754,0.764384,0.764172,0.76599,0.76599,0.767508,0.756126,0.766537,0.75872,0.764037,0.761091,0.763362,0.762768,0.766262,0.766262,0.769937,0.761934,0.770131,0.770659,0.770609,0.772793,0.768863,0.770582,0.769513,0.769513,0.768116,0.757634,0.767051,0.766555,0.766267,0.765886,0.767027,0.767981,0.766488,0.766488,0.769222,0.756252,0.76928,0.767288,0.76818,0.769077,0.767599,0.768919,0.768918,0.768918
bene2_roc_auc_g2,0.773295,0.773295,0.773295,0.773295,0.773295,0.773295,0.773295,0.773295,0.773194,0.773194,0.771066,0.759129,0.771131,0.769283,0.768145,0.767009,0.772394,0.774078,0.769033,0.769033,0.770211,0.750833,0.767061,0.760285,0.766643,0.761857,0.763866,0.762821,0.765062,0.765062,0.772309,0.75276,0.772539,0.772594,0.770982,0.77057,0.771964,0.772542,0.772049,0.772049,0.772434,0.763504,0.772012,0.771408,0.772985,0.773864,0.769669,0.768707,0.769614,0.769614,0.772245,0.767165,0.771718,0.769895,0.771121,0.773035,0.770406,0.770904,0.770023,0.770023,0.772771,0.76309,0.771735,0.769427,0.771198,0.770368,0.769735,0.770878,0.771917,0.771917,0.772385,0.753582,0.76818,0.761215,0.767141,0.763205,0.767551,0.765734,0.766596,0.766596,0.770868,0.753799,0.770842,0.764438,0.767622,0.763372,0.766561,0.762803,0.766023,0.766023,0.774691,0.768026,0.775642,0.778091,0.775274,0.777268,0.77416,0.775654,0.775512,0.775512,0.772747,0.763813,0.773589,0.771881,0.771162,0.770602,0.770561,0.771204,0.771724,0.771724,0.773091,0.760901,0.773504,0.773841,0.773904,0.775625,0.773597,0.775472,0.774827,0.774827
bene2_roc_auc_g3,0.774968,0.774968,0.774997,0.774997,0.774968,0.774968,0.774997,0.774997,0.774919,0.774919,0.775039,0.759609,0.775088,0.770805,0.772686,0.7733,0.775547,0.776794,0.772843,0.772843,0.773135,0.755188,0.77124,0.76341,0.771912,0.768676,0.767823,0.767843,0.770226,0.770226,0.777311,0.762228,0.777784,0.776728,0.775326,0.776882,0.776207,0.777724,0.775286,0.775286,0.7

In [ ]:
'''model_list = ['score', 'random', 'unc', 'qbc', 'dw', 'cors', 'density', 'eer', 'lal', 'quire', 'bmdr', 'spal']

namelist=[]
for d in dataset_list:
  for g in range(10):
    namelist.append(f'{d}_total_cost_g{g+1}')
  
dato = performance.loc[performance.index.str.contains('total_internal_cost')].values + performance.loc[performance.index.str.contains('external_cost')].values
print(np.shape(dato))
total_cost = pd.DataFrame(dato,index=namelist, columns = performance.columns)

performance = pd.concat((performance, total_cost))'''

In [ ]:
performance.loc[performance.index.str.contains('bene2_total_internal_cost')]

In [ ]:
#performance.filter(regex=f'.*_r05')
performance.filter(regex=f'score')

In [ ]:
#performance.loc[performance_ranks.index.str.contains('balance|external_cpl')].T

In [ ]:
#testr = performance.filter(regex='r05').filter(regex='spal')
#np.where(testr['spal_wF_r05'] != testr['spal_wT_r05'])

In [ ]:
#testr.iloc[213]

In [ ]:
performance

# Performance and Ranks (AVERAGE SCORES)

## Prepare Data

In [19]:
"""
  Friedman Test with post-hoc test
"""

#metrics_list = ['roc_auc', 'brier', 'h-measure', 'pcc', 'balanced_accuracy', 'f1','total_internal_cost','total_internal_cpl','external_cost','external_cpl', 'total_cost']
#metrics_list = ['roc_auc', 'brier', 'h-measure', 'pcc', 'balanced_accuracy', 'f1','total_internal_cost','total_internal_cpl','external_cost','external_cpl', 'total_cost']
metrics_list = ['roc_auc', 'brier', 'h-measure', 'pcc', 'f1', 'average_precision', 'precision', 'recall', 'fpr', 'fnr', 'fn','fp','bad_ratio_score_accepts', 'bad_ratio_AL_selects','bad_ratio_accepts', 'bad_ratio_rejects','external_cpl', 'total_internal_cpl']
#metrics_list = ['roc_auc', 'brier', 'h-measure', 'pcc', 'f1', 'external_cpl', 'total_internal_cpl']

dataset_list = ["australian", "bene1_nobins","bene2","german","gmsc", "hmeq","lendingclub", "pakdd", "uk", 'thomas']

#############################################

#BUILD SUBSET OF RESULTS, based on metric and dataset choice
#then create a list of best-performing models, selecting each models' best-performing setup – either overall, or within a specified weight setup

# create list of row names (indices), depending on choice of datasets and metrics
index_names = ''
for i in dataset_list:
  for j in metrics_list:
    index_names += f'{i}_{j}|'
index_names = index_names[:len(index_names)-1] #remove last | symbol
print(index_names)


#store subset of performance and rank dataframes (selected lines only)
performance_sub = performance.loc[performance.index.str.contains(index_names)]
#performance_sub = performance.loc[performance.index.str.contains('g1|g2|g3|g4|g5')]
#performance_sub = performance.loc[performance.index.str.contains('g6|g7|g8|g9|g10')]

###############
# AUGMENT DATA SUBSET for running tests

for m in model_list:
  if m == 'score':
    performance_sub['score'] = performance_sub.filter(regex=f'{m}_').mean(axis=1)
  #if m != 'score':
    #performance_sub.loc[:,f'{m}_r05'] = performance.filter(regex=f'{m}_.*_r05').mean(axis=1) 
    #no need to differentiate between wT and wF when r = 0.5, since the dynamic (inversely proportional) sample weights will all be 1
    #(no difference between score and AL-selections in terms of sample weights)


for m in model_list:
  for w in ['wT', 'wF']:
    if m == 'score':
      for r in ["r01","r02","r03","r04","r05"]:
        performance_sub = performance_sub.drop(f'{m}_{w}_{r}', axis=1)
    #if m != 'score':    
      #performance_sub = performance_sub.drop(f'{m}_{w}_r05', axis=1)

performance_sub = performance_sub.reindex(sorted(performance_sub.columns), axis=1)


###############
# BUILD RANKS after augmentation

performance_sub_ranks = performance_sub.rank(ascending=False, axis="columns")
##invert ranks for metrics where less is better
performance_sub_ranks.loc[~performance_sub_ranks.index.str.contains('roc_auc|h-measure|pcc|f1|average_precision|precision|recall|bad_ratio_rejects')] = performance_sub.loc[performance_sub.index.str.contains('brier|cost|cpl')].rank(ascending=True, axis="columns")
#performance_sub_ranks.loc[performance_sub_ranks.index.str.contains('brier|cost|cpl|bad_ratio_accepts|fnr|fpr|fn|fr|bad_ratio_score_accepts|bad_ratio_AL_selects')] = performance_sub.loc[performance_sub.index.str.contains('brier|cost|cpl')].rank(ascending=True, axis="columns")
#performance_sub_ranks = performance_ranks.loc[performance.index.str.contains(index_names)]

#before picking best-performing models for the overall comparison setup, drop any columns (models) that are redundant


performance_sub_ranks.loc['mean_ranks']=(performance_sub_ranks.loc[~performance_sub_ranks.index.str.contains('internal')].mean())
performance_sub_ranks.loc['cost_ranks']=(performance_sub_ranks.loc[performance_sub_ranks.index.str.contains('external_cpl')].mean())



###############
# get best-performing models, determined by average rank over datasets
# three setups: overall best, best by either weight setup
# option to pick best models either by averaging scores except internal cost, or by just picking external cost

#rank_choice = 'mean_ranks'
rank_choice = 'cost_ranks'

best_performers = {}
namelist = ['score']
#namelist = []
for m in model_list:
  #print(m)
  if m != 'score':
    #print(performance_sub_ranks.filter(regex=f'{m}').loc[rank_choice])
    #print(performance_sub_ranks.filter(regex=f'{m}').loc[rank_choice].idxmin())
    namelist.append(performance_sub_ranks.filter(regex=f'{m}').loc[rank_choice].idxmin())
best_performers['overall'] = namelist

for w in ['wT', 'wF']:
  namelist = ['score']
  #namelist = []
  for m in model_list:
    #print(m)
    if m != 'score':
      #print(performance_sub_ranks.filter(regex=f'{m}_{w}').loc[rank_choice])
      #print(performance_sub_ranks.filter(regex=f'{m}_{w}').loc[rank_choice].idxmin())
      namelist.append(performance_sub_ranks.filter(regex=f'{m}_{w}').loc[rank_choice].idxmin())
  best_performers[w] = namelist

for i in best_performers.keys():
  print(best_performers[i])

performance_sub_ranks

australian_roc_auc|australian_brier|australian_h-measure|australian_pcc|australian_f1|australian_average_precision|australian_precision|australian_recall|australian_fpr|australian_fnr|australian_fn|australian_fp|australian_bad_ratio_score_accepts|australian_bad_ratio_AL_selects|australian_bad_ratio_accepts|australian_bad_ratio_rejects|australian_external_cpl|australian_total_internal_cpl|bene1_nobins_roc_auc|bene1_nobins_brier|bene1_nobins_h-measure|bene1_nobins_pcc|bene1_nobins_f1|bene1_nobins_average_precision|bene1_nobins_precision|bene1_nobins_recall|bene1_nobins_fpr|bene1_nobins_fnr|bene1_nobins_fn|bene1_nobins_fp|bene1_nobins_bad_ratio_score_accepts|bene1_nobins_bad_ratio_AL_selects|bene1_nobins_bad_ratio_accepts|bene1_nobins_bad_ratio_rejects|bene1_nobins_external_cpl|bene1_nobins_total_internal_cpl|bene2_roc_auc|bene2_brier|bene2_h-measure|bene2_pcc|bene2_f1|bene2_average_precision|bene2_precision|bene2_recall|bene2_fpr|bene2_fnr|bene2_fn|bene2_fp|bene2_bad_ratio_score_accepts|

,bmdr_wF_r01,bmdr_wF_r02,bmdr_wF_r03,bmdr_wF_r04,bmdr_wF_r05,bmdr_wT_r01,bmdr_wT_r02,bmdr_wT_r03,bmdr_wT_r04,bmdr_wT_r05,cors_wF_r01,cors_wF_r02,cors_wF_r03,cors_wF_r04,cors_wF_r05,cors_wT_r01,cors_wT_r02,cors_wT_r03,cors_wT_r04,cors_wT_r05,density_wF_r01,density_wF_r02,density_wF_r03,density_wF_r04,density_wF_r05,density_wT_r01,density_wT_r02,density_wT_r03,density_wT_r04,density_wT_r05,dw_wF_r01,dw_wF_r02,dw_wF_r03,dw_wF_r04,dw_wF_r05,dw_wT_r01,dw_wT_r02,dw_wT_r03,dw_wT_r04,dw_wT_r05,eer_wF_r01,eer_wF_r02,eer_wF_r03,eer_wF_r04,eer_wF_r05,eer_wT_r01,eer_wT_r02,eer_wT_r03,eer_wT_r04,eer_wT_r05,lal_wF_r01,lal_wF_r02,lal_wF_r03,lal_wF_r04,lal_wF_r05,lal_wT_r01,lal_wT_r02,lal_wT_r03,lal_wT_r04,lal_wT_r05,qbc_wF_r01,qbc_wF_r02,qbc_wF_r03,qbc_wF_r04,qbc_wF_r05,qbc_wT_r01,qbc_wT_r02,qbc_wT_r03,qbc_wT_r04,qbc_wT_r05,quire_wF_r01,quire_wF_r02,quire_wF_r03,quire_wF_r04,quire_wF_r05,quire_wT_r01,quire_wT_r02,quire_wT_r03,quire_wT_r04,quire_wT_r05,random_wF_r01,random_wF_r02,random_wF_r03,random_wF_r04,random_wF_r05,random_wT_r01,random_wT_r02,random_wT_r03,random_wT_r04,random_wT_r05,score,spal_wF_r01,spal_wF_r02,spal_wF_r03,spal_wF_r04,spal_wF_r05,spal_wT_r01,spal_wT_r02,spal_wT_r03,spal_wT_r04,spal_wT_r05,unc_wF_r01,unc_wF_r02,unc_wF_r03,unc_wF_r04,unc_wF_r05,unc_wT_r01,unc_wT_r02,unc_wT_r03,unc_wT_r04,unc_wT_r05
bene2_roc_auc_g1,29.0000,45.000,58.0000,46.000,53.50,106.0000,51.000,72.000,30.000,53.50,26.0000,28.000,41.000,61.000,39.5000,94.000,20.000,11.0000,43.000,39.5000,15.000,25.000,67.0000,82.0000,56.500,100.000,55.0000,63.000,76.0000,56.500,23.000,38.000,22.000,78.0000,74.500,109.000,96.00,44.000,81.000,74.500,33.0000,50.0000,77.000,84.000,69.500,102.0000,101.000,87.0000,85.0000,69.500,36.000,52.0000,86.000,88.000,59.5000,108.000,104.0000,99.0000,93.000,59.5000,37.000,21.0000,68.0000,49.000,65.500,111.000,62.000,73.000,34.000,65.500,8.0000,5.0000,3.000,19.0000,9.500,95.000,2.000,1.0000,4.0000,9.500,32.000,79.0000,71.0000,24.0000,47.5000,105.0000,92.0000,80.000,7.000,47.5000,6.00,13.0000,12.000,27.000,35.000,17.5000,107.000,42.0000,14.0000,16.0000,17.5000,31.0000,64.000,83.0000,91.0000,89.500,110.000,103.0000,98.000,97.000,89.500
bene2_roc_auc_g2,26.0000,19.000,49.0000,60.000,43.50,96.0000,41.000,58.000,47.000,43.50,34.0000,45.000,51.000,61.000,64.5000,81.000,66.000,23.0000,54.000,64.5000,25.000,42.000,48.0000,67.0000,39.500,100.000,71.0000,62.000,55.0000,39.500,30.000,37.000,24.000,68.0000,69.500,97.000,46.00,16.000,75.000,69.500,32.0000,76.0000,82.000,80.000,86.500,109.0000,104.000,99.0000,91.0000,86.500,56.000,57.0000,79.000,88.000,89.5000,108.000,94.0000,98.0000,102.000,89.5000,33.000,29.0000,53.0000,38.000,35.500,110.000,27.000,59.000,28.000,35.500,12.0000,4.0000,9.000,13.0000,6.500,78.000,1.000,2.0000,3.0000,6.500,52.000,50.0000,77.0000,31.0000,73.5000,107.0000,72.0000,84.000,14.000,73.5000,21.00,22.0000,20.000,15.000,18.000,10.5000,105.000,17.0000,5.0000,8.0000,10.5000,63.0000,83.000,85.0000,95.0000,92.500,111.000,106.0000,103.000,101.000,92.500
bene2_roc_auc_g3,26.0000,30.000,45.0000,53.000,50.50,104.0000,74.000,56.000,57.000,50.50,61.0000,54.000,64.000,63.000,58.5000,95.000,55.000,44.0000,46.000,58.5000,49.000,52.000,35.0000,41.0000,27.500,105.000,82.0000,60.000,33.0000,27.500,34.000,37.000,31.000,76.0000,70.500,97.000,48.00,29.000,88.000,70.500,23.0000,84.0000,89.000,81.000,79.500,110.0000,93.000,96.0000,94.0000,79.500,65.000,62.0000,75.000,83.000,86.5000,109.000,78.0000,99.0000,101.000,86.5000,20.000,17.0000,38.0000,32.000,39.500,107.000,25.000,22.000,18.000,39.500,12.0000,4.0000,6.000,16.0000,9.500,100.000,2.000,1.0000,5.0000,9.500,43.000,42.0000,72.0000,36.0000,68.5000,108.0000,90.0000,66.000,24.000,68.5000,47.00,21.0000,11.000,13.000,19.000,14.5000,73.000,8.0000,3.0000,7.0000,14.5000,67.0000,85.000,77.0000,103.0000,91.500,111.000,106.0000,98.000,102.000,91.500
bene2_roc_auc_g4,37.0000,29.000,61.0000,54.000,55.50,108.0000,92.000,52.000,59.000,55.50,57.0000,46.000,58.000,48.000,32.5000,88.000,23.000,31.0000,15.000,32.

In [ ]:
performance_sub[best_performers['overall']]

In [ ]:
performance_sub_ranks.filter(regex='bmdr_wT')

## Run Friedman and Post-Hoc tests for BEST PERFORMERS ONLY

In [20]:
#############################################
#RUN TESTS

friedman_results2 = {}
#nemenyi_results = {}
#conover_results = {}
#holm_results = {}
finner_results = {}
#quade_scores = {}
rankings_dict = {}
pval_dict = {}
test_results = {}

friedman_statistics = pd.DataFrame(columns=['statistic', 'pvalue'], index=metrics_list)

for w in ['overall','wT', 'wF']: #,'wT', 'wF'
  print('\n\n\n------------------------\nSETUP:', w)
  print('best performing models:', best_performers[w])
  
  #nemenyi_results[w] = {}
  #conover_results[w] = {}
  #holm_results[w] = {}
  finner_results[w] = {}

  rankings = pd.DataFrame(columns = best_performers[w], index=metrics_list)
  pvals = pd.DataFrame(columns = best_performers[w], index=metrics_list)

  for s in metrics_list:
    metric_values = []

    for d in dataset_list:
      row_values = []

      for m in best_performers[w]:
        row_values.append(performance_sub[f'{m}'].loc[performance_sub.index.str.contains(f'{d}_{s}')].mean()) #mean over 10 generations
        columns = best_performers[w]
      
      metric_values.append(row_values)

    #invert sign of measurements for metrics where more is better
    #this is done because the friedman test uses the rankdata function, which assigns ranks from lowest to highest (low score = low rank)
    if s not in ['brier','total_internal_cost','total_internal_cpl','external_cost','external_cpl', 'total_cost', 'fnr','fpr','fn','fp','bad_ratio_accepts', 'bad_ratio_AL_selects']:
      metric_values = np.array(metric_values) * -1
    
    print('\n\n------------------------\nMETRIC', s)
    print(pd.DataFrame(metric_values, columns = columns))

    #statistic, pvalue,statistic_uncorr, p_uncorr,ranking,ranking_avg, rank_cmp = friedman_test(*np.transpose(metric_values)) #this test applies the Iman-Davenport correction to
    statistic, pvalue, ranking, ranking_avg, rank_cmp = friedmanchisquare2(*np.transpose(metric_values))

    print(columns)
    print("average_ranks", ranking_avg)
    print("pivot", rank_cmp)

    temp = {'statistic':statistic, 'pvalue':pvalue}
    friedman_statistics.loc[f'{s}'] = temp
    rankings.loc[f'{s}'] = ranking_avg
    

    # POST HOC TEST HOLM
    pivot_scores = {key: rank_cmp[i] for i, key in enumerate(columns)} 
    print(pivot_scores)
    #print(pd.DataFrame(np.transpose(pivot_scores), columns = columns))
    
    #get the name of the best-ranking model to use as control
    ## REDUNDAND FEATURE; post-hoc tests automatically compare against lowest-scoring models when no control classifier is given...
    min_idx = pd.DataFrame(np.transpose(list(pivot_scores.values())), index = pivot_scores.keys()).iloc[:,0].idxmin() 
    
    '''comparisons, z_values, p_values, adj_p_values = holm_test(pivot_scores, control=columns[min_rank])
    adj_p_values = np.asarray(adj_p_values)
    holm_scores = pd.DataFrame({"p": adj_p_values, "sig": adj_p_values < 0.05}, index=comparisons)
    #print(holm_scores)
    holm_results[w][s] = copy.deepcopy(holm_scores)'''

    #comparisons, z_values, p_values, adj_p_values, keys = finner_test(pivot_scores, control=columns[min_rank])
    comparisons, z_values, p_values, adj_p_values = finner_test(pivot_scores, control = min_idx)
    adj_p_values = np.asarray(adj_p_values)
    finner_scores = pd.DataFrame({"p": adj_p_values, "sig": adj_p_values < 0.05}, index=comparisons, )
    print(p_values)
    print(adj_p_values)
    print(finner_scores)
    finner_results[w][s] = copy.deepcopy(finner_scores)

    for m in best_performers[w]:
      try:
        pvals.at[s,m] = np.round(finner_scores.loc[finner_scores.index.str.contains(f'vs {m}')]['p'].values, 4)
      except KeyError:
        pvals.at[s,m] = np.nan


    #nemenyi_results[w][s] = generate_scores(sp.posthoc_nemenyi_friedman, {}, metric_values, columns)
    #conover_results[w][s] = generate_scores(sp.posthoc_conover_friedman, {'p_adjust':'holm-sidak'}, metric_values, columns)
    #quade_scores[s] = generate_scores(sp.posthoc_quade, {}, metric_values, columns)

  print(f'Friedman results\n',friedman_statistics,'\n\n')
  friedman_results2[w] = copy.deepcopy(friedman_statistics)
  pvals = pvals.T.add_suffix('_p-values')
  pvals.loc['Friedman'] = np.round(friedman_statistics['pvalue'].to_numpy().flatten().astype(float),4)

  rankings = rankings.T.add_suffix('_avg-rank')
  rankings.loc['Friedman'] = np.round(friedman_statistics['statistic'].to_numpy().flatten().astype(float),4)
  rankings['AvgRank'] = np.round(rankings.mean(axis=1),2)
  rankings['AvgRank'].loc['Friedman'] = np.nan
  rankings['HighScore'] = rankings['AvgRank'].rank()
  rankings_dict[w] = copy.deepcopy(rankings)

  pval_dict[w] = copy.deepcopy(pvals)
  combined_results = pd.concat((rankings, pvals), axis=1)

  test_results[w] = combined_results.reindex(sorted(combined_results.columns), axis=1)







------------------------
SETUP: overall
best performing models: ['score', 'random_wT_r04', 'unc_wF_r01', 'qbc_wT_r04', 'dw_wF_r03', 'cors_wF_r01', 'density_wT_r04', 'eer_wF_r02', 'lal_wF_r02', 'quire_wT_r04', 'bmdr_wT_r04', 'spal_wF_r04']


------------------------
METRIC roc_auc
      score  random_wT_r04  unc_wF_r01  qbc_wT_r04  dw_wF_r03  cors_wF_r01  \
0 -0.918350      -0.915087   -0.916759   -0.920703  -0.919958    -0.916236   
1 -0.748293      -0.753867   -0.747846   -0.745927  -0.743562    -0.750068   
2 -0.778209      -0.777812   -0.776355   -0.779185  -0.777597    -0.777840   
3 -0.773833      -0.776912   -0.777436   -0.777290  -0.779121    -0.774126   
4 -0.839645      -0.843481   -0.841919   -0.843312  -0.843139    -0.842159   
5 -0.780813      -0.783517   -0.780313   -0.784620  -0.782407    -0.778114   
6 -0.620482      -0.625864   -0.625332   -0.627058  -0.622621    -0.620470   
7 -0.594101      -0.590028   -0.591857   -0.594965  -0.596153    -0.594124   
8 -0.711233   

In [21]:
test_results['overall']

,AvgRank,HighScore,average_precision_avg-rank,average_precision_p-values,bad_ratio_AL_selects_avg-rank,bad_ratio_AL_selects_p-values,bad_ratio_accepts_avg-rank,bad_ratio_accepts_p-values,bad_ratio_rejects_avg-rank,bad_ratio_rejects_p-values,bad_ratio_score_accepts_avg-rank,bad_ratio_score_accepts_p-values,brier_avg-rank,brier_p-values,external_cpl_avg-rank,external_cpl_p-values,f1_avg-rank,f1_p-values,fn_avg-rank,fn_p-values,fnr_avg-rank,fnr_p-values,fp_avg-rank,fp_p-values,fpr_avg-rank,fpr_p-values,h-measure_avg-rank,h-measure_p-values,pcc_avg-rank,pcc_p-values,precision_avg-rank,precision_p-values,recall_avg-rank,recall_p-values,roc_auc_avg-rank,roc_auc_p-values,total_internal_cpl_avg-rank,total_internal_cpl_p-values
score,6.11,2.0,9.7,[0.0],1.4,[],1.2,[],1.6,[],6.3,[0.0027],10.5,[0.0],7.2,[0.6626],7.7,[0.1541],5.8,[0.4128],5.8,[0.4128],7.6,[0.0906],7.6,[0.0906],7.3,[0.0423],7.8,[0.0191],7.7,[0.2149],5.8,[0.4128],7.6,[0.0677],1.4,[]
random_wT_r04,6.15,3.0,5.2,[0.0662],4.8,[0.0384],8.0,[0.0001],7.7,[0.0004],8.5,[0.0],4.7,[0.1928],5.2,[0.7098],5.2,[0.5694],8.4,[0.0345],8.4,[0.0345],4.7,[0.6989],4.7,[0.6989],4.7,[0.4201],4.2,[0.6116],4.7,[0.9505],8.4,[0.0345],5.4,[0.2834],7.8,[0.0002]
unc_wF_r01,6.27,4.0,8.6,[0.0002],9.9,[0.0],3.5,[0.1678],4.1,[0.1323],1.3,[],7.9,[0.0037],6.4,[0.6626],6.0,[0.3485],6.7,[0.2201],6.7,[0.2201],6.1,[0.2551],6.1,[0.2551],8.7,[0.0111],7.2,[0.0423],5.7,[0.6093],6.7,[0.2201],7.5,[0.0677],3.7,[0.1678]
qbc_wT_r04,5.73,1.0,2.1,[],6.3,[0.0043],9.9,[0.0],9.0,[0.0],8.1,[0.0001],2.6,[],4.6,[],4.2,[],7.0,[0.2108],7.0,[0.2108],5.2,[0.5256],5.2,[0.5256],3.4,[],4.2,[0.6116],4.6,[],7.0,[0.2108],3.5,[],9.3,[0.0]
dw_wF_r03,7.03,11.0,6.0,[0.0244],7.3,[0.0006],7.9,[0.0001],7.7,[0.0004],6.7,[0.0013],4.8,[0.188],6.9,[0.6626],8.7,[0.0563],6.7,[0.2201],6.7,[0.2201],6.8,[0.146],6.8,[0.146],6.9,[0.0542],7.0,[0.0423],7.9,[0.2149],6.7,[0.2201],7.2,[0.0677],7.9,[0.0002]
cors_wF_r01,6.48,7.0,9.9,[0.0],8.9,[0.0],2.9,[0.2917],2.9,[0.4201],2.0,[0.6642],10.0,[0.0],7.3,[0.6626],7.1,[0.1634],6.8,[0.2201],6.8,[0.2201],6.4,[0.2062],6.4,[0.2062],8.3,[0.013],6.9,[0.0423],6.5,[0.3485],6.8,[0.2201],7.9,[0.0677],2.8,[0.3853]
density_wT_r04,6.29,5.0,4.8,[0.094],4.5,[0.0545],7.1,[0.0004],7.0,[0.0011],7.6,[0.0002],5.5,[0.0978],6.7,[0.6626],5.9,[0.3485],8.6,[0.0345],8.6,[0.0345],4.5,[0.7565],4.5,[0.7565],5.4,[0.2336],5.2,[0.2834],5.1,[0.8221],8.6,[0.0345],6.6,[0.0844],7.0,[0.0007]
eer_wF_r02,6.87,10.0,6.3,[0.0201],9.9,[0.0],7.0,[0.0004],7.2,[0.0009],4.5,[0.0518],5.9,[0.0733],7.3,[0.6626],7.2,[0.1634],5.5,[0.4862],5.5,[0.4862],7.4,[0.0906],7.4,[0.0906],7.2,[0.0423],7.2,[0.0423],7.7,[0.2149],5.5,[0.4862],7.8,[0.0677],7.1,[0.0006]
lal_wF_r02,6.85,9.0,8.3,[0.0003],8.2,[0.0001],5.7,[0.0064],5.9,[0.0094],5.1,[0.0225],7.6,[0.0053],6.5,[0.6626],7.7,[0.1541],4.5,[0.8524],4.5,[0.8524],8.5,[0.0286],8.5,[0.0286],7.6,[0.0333],9.1,[0.0035],7.9,[0.2149],4.5,[0.8524],7.2,[0.0677],6.0,[0.0053]
quire_wT_r04,6.54,8.0,5.1,[0.0689],5.6,[0.0126],8.3,[0.0],8.1,[0.0002],10.2,[0.0],5.9,[0.0733],5.8,[0.6626],6.1,[0.3485],4.8,[0.7436],4.8,[0.7436],7.6,[0.0906],7.6,[0.0906],5.5,[0.2303],6.9,[0.0423],6.9,[0.2637],4.8,[0.7436],5.4,[0.2834],8.3,[0.0001]


In [22]:
for m in model_list:
  print(performance_sub_ranks.loc['cost_ranks'].filter(regex=f'{m}').idxmin())

score
random_wT_r04
unc_wF_r01
qbc_wT_r04
dw_wF_r03
cors_wF_r01
density_wT_r04
eer_wF_r02
lal_wF_r02
quire_wT_r04
bmdr_wT_r04
spal_wF_r04


In [ ]:
friedman_results2['overall'] #uncorrected pvalues (without thomas)

,statistic,pvalue
roc_auc,1.419365,0.176099
brier,5.765948,0.0
h-measure,1.973738,0.038962
pcc,3.346508,0.000595
f1,1.31746,0.226099
average_precision,5.820359,0.0
precision,1.627581,0.102481
recall,2.226448,0.018558
fpr,2.38535,0.011521
fnr,2.226448,0.018558


In [ ]:
friedman = np.round(friedman_results2['overall'].to_numpy().flatten().astype(float),4)
friedman

In [ ]:
friedman_results2['overall'] #iman davenport pvalues (with thomas)

In [ ]:
finner_results['overall']

In [ ]:
holm_results['overall']

In [ ]:
nemenyi_results['overall']['brier']

In [ ]:
'''for k1 in nemenyi_results.keys():
  for k2 in nemenyi_results[k1].keys():
    pairwise_plotter(nemenyi_results[k1][k2])'''

#pairwise_plotter(nemenyi_results['overall']['total_internal_cpl'])
pairwise_plotter(conover_results['overall']['roc_auc'])

In [ ]:
sp.sign_plot(nemenyi_results['overall']['brier'])

## Run Friedman Tests for 2x60 models (average SCORES)

In [ ]:
"""
  Runs test comparing ALL models for each weight setup
"""


#model_list = ['score', 'random', 'unc', 'qbc', 'dw', 'cors', 'density', 'eer', 'lal', 'quire', 'bmdr', 'spal']
#metrics_list = ['roc_auc', 'brier', 'h-measure', 'pcc', 'balanced_accuracy', 'f1','total_internal_cost','total_internal_cpl','external_cost','external_cpl']
metrics_list = ['roc_auc', 'brier', 'h-measure', 'pcc','balanced_accuracy', 'f1', 'total_internal_cpl','external_cpl']  #'brier', 'h-measure', 'pcc',
dataset_list = ["bene2", "bene1_nobins","gmsc", "uk", "lendingclub", "hmeq", "pakdd", "australian", "german"] #,"thomas", 
#dataset_list = ["bene2", "bene1_nobins","gmsc", "uk", "hmeq", "australian", "german"]
#dataset_list = ["lendingclub", "thomas", "pakdd"]


friedman_results = {'wT':{}, 'wF':{}}
nemenyi_scores = {}
quade_scores = {}

for w in ['wT', 'wF']:
  friedman_results[w] = {}
  friedman_statistics = pd.DataFrame(columns=['statistic', 'pvalue'], index=metrics_list)

  for s in metrics_list:
    metric_values = []

    for d in dataset_list:
      row_values = []
      columns = []

      for m in model_list:
        row_values.extend(performance.filter(regex=f'{m}_{w}').loc[performance.index.str.contains(f'{d}_{s}')].mean().tolist())
        columns.extend(list(performance.filter(regex=f'{m}_{w}').columns))
      
      metric_values.append(row_values)

    #invert sign of measurements for metrics where more is better – needed because friedmanchisquare() uses scipy.stats.rankdata(), which ranks in ascending fashion (low value -> low rank)
    if s not in ['brier','total_internal_cost','total_internal_cpl','external_cost','external_cpl']:
      metric_values = np.array(metric_values) * -1

    print('\n\n\n------------------------\nMETRIC', s)

    #statistic, pvalue,statistic_uncorr, p_uncorr,ranking,ranking_avg, rank_cmp = friedman_test(*np.transpose(metric_values))
    statistic, pvalue, ranking, ranking_avg, rank_cmp = friedmanchisquare2(*np.transpose(metric_values))

    print('\n-------------------------------\nscipy Friedman MODDED\n-------------------------------\n')
    for pr in [statistic, pvalue, ranking_avg, rank_cmp]: #, ranking,ranking_avg, rank_cmp]:
      print(pr)

    temp = {'statistic':statistic, 'pvalue':pvalue}
    friedman_statistics.loc[f'{s}'] = temp



    # POST HOC TEST HOLM
    ranks = {key: rank_cmp[i] for i, key in enumerate(columns)} 
    min_rank = pd.DataFrame(np.transpose(list(ranks.values())), index = ranks.keys()).iloc[:,0].argmin()
    
    comparisons, z_values, p_values, adj_p_values = holm_test(ranks, control=columns[min_rank])
    adj_p_values = np.asarray(adj_p_values)
    holm_scores = pd.DataFrame({"p": adj_p_values, "sig": adj_p_values < 0.05}, index=comparisons)
    print(holm_scores)    


    comparisons, z_values, p_values, adj_p_values = finner_test(pivot_scores, control=columns[min_rank])
    adj_p_values = np.asarray(adj_p_values)
    finner_scores = pd.DataFrame({"p": adj_p_values, "sig": adj_p_values < 0.05}, index=comparisons)
    print(finner_scores)
    #finner_results[w][s] = copy.deepcopy(finner_scores)


    #nemenyi_scores[s] = generate_scores(sp.posthoc_nemenyi_friedman, {}, metric_values, columns)
    #quade_scores[s] = generate_scores(sp.posthoc_quade, {}, metric_values, columns)


  print(np.shape(metric_values))
  print(f'Friedman results for {w}\n',friedman_statistics,'\n\n')
  friedman_results[w][r] = friedman_statistics

'''for key in nemenyi_scores.keys():
  pairwise_plotter(nemenyi_scores[key])'''





------------------------
METRIC roc_auc

-------------------------------
scipy Friedman MODDED
-------------------------------

275.40295307818246
1.1040457182485505e-29
[24.77777777777778, 24.77777777777778, 24.77777777777778, 24.77777777777778, 24.77777777777778, 51.111111111111114, 34.666666666666664, 25.88888888888889, 17.666666666666668, 21.444444444444443, 54.22222222222222, 38.0, 31.88888888888889, 28.77777777777778, 22.88888888888889, 55.111111111111114, 32.111111111111114, 21.555555555555557, 12.88888888888889, 16.0, 55.44444444444444, 42.0, 28.555555555555557, 25.555555555555557, 22.666666666666668, 47.888888888888886, 38.111111111111114, 24.555555555555557, 16.88888888888889, 17.444444444444443, 53.77777777777778, 35.22222222222222, 24.555555555555557, 16.77777777777778, 15.333333333333334, 54.22222222222222, 39.888888888888886, 29.666666666666668, 22.22222222222222, 21.77777777777778, 54.0, 38.111111111111114, 28.666666666666668, 23.88888888888889, 18.444444444444443, 47

ValueError: ignored

In [ ]:
columns = []
columns.append(performance.filter(regex='bmdr_wT').columns)
type(columns)

list

In [ ]:
 performance.filter(regex=f'{w}_{r}').columns

Index(['bmdr_wF_r05', 'cors_wF_r05', 'density_wF_r05', 'dw_wF_r05',
       'eer_wF_r05', 'lal_wF_r05', 'qbc_wF_r05', 'quire_wF_r05',
       'random_wF_r05', 'score_wF_r05', 'spal_wF_r05', 'unc_wF_r05'],
      dtype='object')

In [ ]:
pd.DataFrame(np.transpose(list(ranks.values())), index = ranks.keys())

,0
score_wF_r05,2.668498
random_wF_r05,3.780372
unc_wF_r05,3.520934
qbc_wF_r05,3.261497
dw_wF_r05,3.854496
cors_wF_r05,3.632122
density_wF_r05,3.039122
eer_wF_r05,3.150310
lal_wF_r05,4.521621
quire_wF_r05,2.446123


In [ ]:
ranks

{'bmdr_wF_r05': 2.816747403312185,
 'cors_wF_r05': 3.632121651639396,
 'density_wF_r05': 3.0391221983105146,
 'dw_wF_r05': 3.8544964466377265,
 'eer_wF_r05': 3.15030959580968,
 'lal_wF_r05': 4.521620831632717,
 'qbc_wF_r05': 3.261496993308845,
 'quire_wF_r05': 2.446122744981634,
 'random_wF_r05': 3.780371514971616,
 'score_wF_r05': 2.6684975399799646,
 'spal_wF_r05': 3.780371514971616,
 'unc_wF_r05': 3.5209342541402306}

In [ ]:
friedman_results

{'wF': {'r01':                     statistic    pvalue
  roc_auc             11.725275  0.384644
  brier               21.263736  0.030739
  h-measure            5.241758  0.918895
  pcc                  7.263736  0.777341
  balanced_accuracy   11.021978  0.441424
  f1                   9.461538   0.57937
  total_internal_cpl  40.538462  0.000029
  external_cpl        11.021978  0.441424,
  'r02':                     statistic    pvalue
  roc_auc              7.967033  0.716254
  brier               21.857143  0.025504
  h-measure            6.956044  0.802632
  pcc                  8.868132  0.634063
  balanced_accuracy    9.725275  0.555255
  f1                   5.285714  0.916544
  total_internal_cpl  43.175824   0.00001
  external_cpl         9.725275  0.555255,
  'r03':                     statistic    pvalue
  roc_auc              6.802198  0.814868
  brier               19.769231  0.048608
  h-measure            4.296703   0.96042
  pcc                  9.527473  0.573323
  bal

## run separate tests for each combination of weights and ratio

In [ ]:
#Friedman Test with post-hoc test
#model_list = ['score', 'random', 'unc', 'qbc', 'dw', 'cors', 'density', 'eer', 'lal', 'quire', 'bmdr', 'spal']
#metrics_list = ['roc_auc', 'brier', 'h-measure', 'pcc', 'balanced_accuracy', 'f1','total_internal_cost','total_internal_cpl','external_cost','external_cpl']
metrics_list = ['roc_auc', 'brier', 'h-measure', 'pcc','balanced_accuracy', 'f1', 'total_internal_cpl','external_cpl']  #'brier', 'h-measure', 'pcc',
dataset_list = ["bene2", "bene1_nobins","gmsc", "uk", "lendingclub", "hmeq", "pakdd"] #"australian", "german", "thomas", 
#dataset_list = ["bene2", "bene1_nobins","gmsc", "uk", "hmeq", "australian", "german"]
#dataset_list = ["lendingclub", "thomas", "pakdd"]


friedman_results = {'wT':{}, 'wF':{}}
nemenyi_scores = {}
quade_scores = {}

average_ratios = False #this feature does not work correctly, if this is enabled it is unnecessary to loop over ratios. 

for w in ['wT', 'wF']:
  friedman_results[w] = {}
  
  for r in ["r01","r02","r03","r04","r05"]:
    friedman_statistics = pd.DataFrame(columns=['statistic', 'pvalue'], index=metrics_list)

    for s in metrics_list:
      metric_values = []

      for d in dataset_list:
        row_values = []
        if average_ratios:
          columns = model_list
        else:
          columns = []

        for m in model_list:
          if average_ratios:
            row_values.append(performance.filter(regex=f'{m}_{w}').loc[performance.index.str.contains(f'{d}_{s}')].mean().mean())
          else:
            row_values.extend(performance.filter(regex=f'{m}_{w}_{r}').loc[performance.index.str.contains(f'{d}_{s}')].mean().tolist())
            columns.append(f'{m}_{w}_{r}')
        
        metric_values.append(row_values)

      #invert sign of measurements for metrics where more is better – needed because friedmanchisquare() uses scipy.stats.rankdata(), which ranks in ascending fashion (low value -> low rank)
      if s not in ['brier','total_internal_cost','total_internal_cpl','external_cost','external_cpl']:
        metric_values = np.array(metric_values) * -1

      print('\n\n\n------------------------\nMETRIC', s)

      #statistic, pvalue,statistic_uncorr, p_uncorr,ranking,ranking_avg, rank_cmp = friedman_test(*np.transpose(metric_values))
      statistic, pvalue, ranking, ranking_avg, rank_cmp = friedmanchisquare2(*np.transpose(metric_values))

      print('\n-------------------------------\nscipy Friedman MODDED\n-------------------------------\n')
      for pr in [statistic, pvalue, ranking_avg, rank_cmp]: #, ranking,ranking_avg, rank_cmp]:
        print(pr)

      print('\n-------------------------------\nCustom STAC\n-------------------------------\n')
      print(friedman_test(*np.transpose(metric_values)))

      temp = {'statistic':statistic, 'pvalue':pvalue}
      friedman_statistics.loc[f'{s}'] = temp



      # POST HOC TEST HOLM
      ranks = {key: rank_cmp[i] for i, key in enumerate(columns)} 
      print('ranking_avg', ranking_avg)
      print(pd.DataFrame(np.transpose(list(ranks.values())), index = ranks.keys()).iloc[:,0])
      min_rank = pd.DataFrame(np.transpose(list(ranks.values())), index = ranks.keys()).iloc[:,0].argmin()
      
      comparisons, z_values, p_values, adj_p_values = holm_test(ranks, control=columns[min_rank])
      adj_p_values = np.asarray(adj_p_values)
      holm_scores = pd.DataFrame({"p": adj_p_values, "sig": adj_p_values < 0.05}, index=comparisons)
      print(holm_scores)    

      #nemenyi_scores[s] = generate_scores(sp.posthoc_nemenyi_friedman, {}, metric_values, columns)
      #quade_scores[s] = generate_scores(sp.posthoc_quade, {}, metric_values, columns)


    print(np.shape(metric_values))
    print(f'Friedman results for {w}\n',friedman_statistics,'\n\n')
    friedman_results[w][r] = friedman_statistics

'''for key in nemenyi_scores.keys():
  pairwise_plotter(nemenyi_scores[key])'''





------------------------
METRIC roc_auc

-------------------------------
scipy Friedman MODDED
-------------------------------

25.571428571428612
0.0075125990565410835
[1.4285714285714286, 5.857142857142857, 8.0, 7.714285714285714, 8.0, 6.428571428571429, 6.142857142857143, 9.285714285714286, 7.428571428571429, 4.428571428571429, 7.714285714285714, 5.571428571428571]
[0.7412493166611012, 3.0391221983105146, 4.1509961733021665, 4.002746309969947, 4.1509961733021665, 3.3356219249749555, 3.1873720616427352, 4.818120558297158, 3.8544964466377265, 2.297872881649414, 4.002746309969947, 2.8908723349782943]

-------------------------------
Custom STAC
-------------------------------

(2.9833333333333334, 0.002844721456970456, 25.571428571428573, 0.0075125990565411945, [[1.0, 6.0, 12.0, 10.0, 3.0, 2.0, 7.0, 11.0, 9.0, 5.0, 8.0, 4.0], [3.0, 8.0, 7.0, 9.0, 11.0, 1.0, 5.0, 2.0, 4.0, 10.0, 6.0, 12.0], [1.0, 7.0, 10.0, 6.0, 11.0, 8.0, 2.0, 9.0, 12.0, 3.0, 5.0, 4.0], [1.0, 2.0, 10.0, 4.0, 9.0, 7.

'for key in nemenyi_scores.keys():\n  pairwise_plotter(nemenyi_scores[key])'